In [3]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(dplyr))
set.seed(43)

In [2]:
t2dggi_gws <- fread("/lustre/groups/itg/teams/zeggini/users/archit.singh/lambda/summary_statistics/T2D/T2DGGI/GWS_single_gc_t2dggi.txt", header = TRUE)
diamante18_gws <- fread("/lustre/groups/itg/teams/zeggini/users/archit.singh/lambda/summary_statistics/T2D/Mahajan.NatGenet2018b.T2D.European.undajusted.BMI/GWS_single_gc_DIAMANTE18.txt", header = TRUE)
diamante22_gws <- fread("/lustre/groups/itg/teams/zeggini/users/archit.singh/lambda/summary_statistics/T2D/Mahajan.NatGen2022.DIAMANTE-EUR.sumstat/GWS_single_gc_DIAMANTE22.txt", header = TRUE)

In [ ]:
#renaming appropriate columns
diamante18_gws <- diamante18_gws %>% rename(beta = Beta)
dim(diamante18_gws)
colnames(diamante18_gws)[colnames(diamante18_gws) == "Chr"] <- "CHR"

dim(diamante22_gws)
colnames(diamante22_gws)[colnames(diamante22_gws) == "chromosome(b37)"] <- "CHR"
diamante22_gws <- diamante22_gws %>% rename(beta = `Fixed-effects_beta`)

dim(t2dggi_gws)
t2dggi_gws$CHR <- sub("^chr(\\d+):.*$", "\\1", t2dggi_gws$MarkerName)
t2dggi_gws <- t2dggi_gws %>% rename(beta = Effect)

In [ ]:
calculate_replication <- function(df1, df2, num_simulations = 100000, alpha = 0.0023) {
  # Check if necessary columns are present
  if (!all(c("CHR", "id", "beta") %in% colnames(df1)) || !all(c("CHR", "id", "beta") %in% colnames(df2))) {
    stop("Both dataframes must contain 'CHR', 'id', and 'beta' columns.")
  }
  
  # Initialize a results list to store replication averages for each chromosome
  replication_results <- list()
  
  # Loop through each chromosome in df1
  for (chr in unique(df1$CHR)) {
    # SNPs on the current chromosome
    df1_chr <- df1[df1$CHR == chr, ]
    
    # Number of SNPs on this chromosome
    x <- nrow(df1_chr)
    
    # Calculate the number of replicated SNPs in df2
    replicated_snps <- sum(
      df1_chr$id %in% df2$id &
        sign(df1_chr$beta[df1_chr$id %in% df2$id]) == sign(df2$beta[match(df1_chr$id[df1_chr$id %in% df2$id], df2$id)])
    )
    
    # Calculate the proportion of replication
    replication_proportion <- replicated_snps / x
    
    # Subset df1 without the current chromosome (LOCO approach)
    df1_loco <- df1[df1$CHR != chr, ]
    
    # Initialize a vector to store results of the simulations
    simulation_results <- numeric(num_simulations)
    
    # Perform simulations
    for (i in seq_len(num_simulations)) {
      # Randomly sample 'x' SNPs from the LOCO data
      sampled_snps <- df1_loco[sample(nrow(df1_loco), x, replace = FALSE), ]
      
      # Count the number of sampled SNPs replicated in df2
      simulation_results[i] <- sum(
        sampled_snps$id %in% df2$id &
          sign(sampled_snps$beta[sampled_snps$id %in% df2$id]) ==
          sign(df2$beta[match(sampled_snps$id[sampled_snps$id %in% df2$id], df2$id)])
      )
    }
    
    # Calculate the average replication from the simulations
    average_simulation <- mean(simulation_results)
    
    # Calculate a two-tailed p-value and ensure it's bounded between 0 and 1
    #p_value <- 2 * min(
    # mean(simulation_results >= replicated_snps), # Probability of observing replicated_snps or more
    #mean(simulation_results <= replicated_snps)  # Probability of observing replicated_snps or fewer
    #)
    #p_value <- max(0, min(1, p_value)) # Bound p-value between 0 and 1
    p_value_higher <- mean(simulation_results >= replicated_snps)
    p_value_lower <- 1-p_value_higher  
    final_p_value <- min(p_value_lower, p_value_higher)
    # Determine if the actual replication rate is significantly different from the simulations
    #significant_lower <- ifelse(p_value_lower < alpha, TRUE, FALSE)
    #significant_higher <- ifelse(p_value_higher < alpha, TRUE, FALSE)
    
    # Store the results
    replication_results[[chr]] <- list(
      chromosome = chr,
      replicated_snps = replicated_snps,
      average_simulation = average_simulation,
      #p_value = p_value,
      p_value_higher = p_value_higher,
      p_value_lower = p_value_lower,
      final_p_value = final_p_value,
      #significant_lower = significant_lower,
      #significant_higher = significant_higher,
      replication_proportion = replication_proportion
    )
  }
  
  # Convert results to a data frame
  results_df <- do.call(rbind, lapply(replication_results, as.data.frame))
  return(results_df)
}


In [ ]:
#Assessing rate of replication before correction
results_diamante22_t2dggi <- calculate_replication(diamante22_gws, t2dggi_gws)
print(results_diamante22_t2dggi)

results_diamante18_t2dggi <- calculate_replication(diamante18_gws, t2dggi_gws)
print(results_diamante18_t2dggi)

results_diamante18_diamante22 <- calculate_replication(diamante18_gws, diamante22_gws)
print(results_diamante18_diamante22)

#writing results to file
#fwrite(results_diamante18_diamante22, "C:/Users/ArchitSingh/Desktop/lambda_project/data_from_lustre/analysis_results/rate_of_replication/diamante18-diamante22.txt", sep = "\t")
#fwrite(results_diamante18_t2dggi, "C:/Users/ArchitSingh/Desktop/lambda_project/data_from_lustre/analysis_results/rate_of_replication/diamante18-t2dggi.txt", sep = "\t")
#fwrite(results_diamante22_t2dggi, "C:/Users/ArchitSingh/Desktop/lambda_project/data_from_lustre/analysis_results/rate_of_replication/diamante22-t2dggi.txt", sep = "\t")


In [ ]:
#Assessing rate of replication after GC correction
#correcting the p-values with the new lambda values; check jupyter notebook titled "getting_variants_with_MAF_morethan_0.005"
#DIAMANTE-18
chisq <- qchisq(as.numeric(diamante18_gws$single_gc_p_values),1,lower.tail=FALSE)
lambda <- c(1.387) #rounding off to 3 digits after decimal point
newchisq <- chisq/lambda
newpvalues <- pchisq(newchisq, df=1,lower.tail=FALSE)
diamante18_gws$double_gc_p_values <- newpvalues

#DIAMANTE-22
chisq <- qchisq(as.numeric(diamante22_gws$single_gc_p_values),1,lower.tail=FALSE)
lambda <- c(1.402) 
newchisq <- chisq/lambda
newpvalues <- pchisq(newchisq, df=1,lower.tail=FALSE)
diamante22_gws$double_gc_p_values <- newpvalues

#identifying variants significant after GC correction
diamante18_gws_GC <- subset(diamante18_gws, diamante18_gws$double_gc_p_values <= 0.00000005)
diamante22_gws_GC <- subset(diamante22_gws, diamante22_gws$double_gc_p_values <= 0.00000005)

#running simulation
results_diamante22_t2dggi_GC <- calculate_replication(diamante22_gws_GC, t2dggi_gws)
print(results_diamante22_t2dggi_GC)

results_diamante18_t2dggi_GC <- calculate_replication(diamante18_gws_GC, t2dggi_gws)
print(results_diamante18_t2dggi_GC)

results_diamante18_diamante22_GC <- calculate_replication(diamante18_gws_GC, diamante22_gws)
print(results_diamante18_diamante22_GC)

In [ ]:
#Assessing rate of replication after LDSR intercept correction
#Correcting the uncorrected P-values using the LDSR-intercept
intercept1 <- c(1.097)
chisq <- qchisq(as.numeric(diamante18_gws$single_gc_p_values),1,lower.tail=FALSE)
newchisq_intercept <- chisq/intercept1
newpvalues <- pchisq(newchisq_intercept, df=1,lower.tail=FALSE)
diamante18_gws$intercept_adj_p_values <- newpvalues

intercept2 <- c(1.0918)
chisq <- qchisq(as.numeric(diamante22_gws$single_gc_p_values),1,lower.tail=FALSE)
newchisq_intercept <- chisq/intercept2
newpvalues <- pchisq(newchisq_intercept, df=1,lower.tail=FALSE)
diamante22_gws$intercept_adj_p_values <- newpvalues

#identifying variants significant after LDSR intercept correction
diamante18_gws_LDSR <- subset(diamante18_gws, diamante18_gws$intercept_adj_p_values <= 0.00000005)
diamante22_gws_LDSR <- subset(diamante22_gws, diamante22_gws$intercept_adj_p_values <= 0.00000005)

#running simulation
results_diamante22_t2dggi_LDSR <- calculate_replication(diamante22_gws_LDSR, t2dggi_gws)
print(results_diamante22_t2dggi_LDSR)

results_diamante18_t2dggi_LDSR <- calculate_replication(diamante18_gws_LDSR, t2dggi_gws)
print(results_diamante18_t2dggi_LDSR)

results_diamante18_diamante22_LDSR <- calculate_replication(diamante18_gws_LDSR, diamante22_gws)
print(results_diamante18_diamante22_LDSR)

In [ ]:
########################## Calculating replication proportions for simulations #################
uncorr_diam18_diam22 <- results_diamante18_diamante22
uncorr_diam18_t2dggi <- results_diamante18_t2dggi
uncorr_diam22_t2dggi <- results_diamante22_t2dggi
gc_diam22_t2dggi <- results_diamante22_t2dggi_GC
ldsr_diam22_t2dggi <- results_diamante22_t2dggi_LDSR

uncorr_diam18_diam22 <- uncorr_diam18_diam22 %>% rename(CHR = chromosome)
uncorr_diam18_t2dggi <- uncorr_diam18_t2dggi %>% rename(CHR = chromosome)
uncorr_diam22_t2dggi <- uncorr_diam22_t2dggi %>% rename(CHR = chromosome)
gc_diam22_t2dggi <- gc_diam22_t2dggi %>% rename(CHR = chromosome)
ldsr_diam22_t2dggi <- ldsr_diam22_t2dggi %>% rename(CHR = chromosome)

gw_significant_counts_diamante_18 <- diamante18_gws %>%
  group_by(CHR) %>%
  summarise(total_gw_snps = n(), .groups = "drop")

gw_significant_counts_diamante_22 <- diamante22_gws %>%
  group_by(CHR) %>%
  summarise(total_gw_snps = n(), .groups = "drop")

uncorr_diam18_diam22 <- uncorr_diam18_diam22 %>%
  left_join(gw_significant_counts_diamante_18, by = "CHR") %>%
  mutate(simulation_replication_proportion = average_simulation / total_gw_snps)

uncorr_diam18_t2dggi <- uncorr_diam18_t2dggi %>%
  left_join(gw_significant_counts_diamante_18, by = "CHR") %>%
  mutate(simulation_replication_proportion = average_simulation / total_gw_snps)

uncorr_diam22_t2dggi <- uncorr_diam22_t2dggi %>%
  left_join(gw_significant_counts_diamante_22, by = "CHR") %>%
  mutate(simulation_replication_proportion = average_simulation / total_gw_snps)

gc_diam22_t2dggi <- gc_diam22_t2dggi %>%
  left_join(gw_significant_counts_diamante_22, by = "CHR") %>%
  mutate(
    simulation_replication_proportion = average_simulation / total_gw_snps,
    real_replication_proportion = replicated_snps / total_gw_snps
  )

ldsr_diam22_t2dggi <- ldsr_diam22_t2dggi %>%
  left_join(gw_significant_counts_diamante_22, by = "CHR") %>%
  mutate(
    simulation_replication_proportion = average_simulation / total_gw_snps,
    real_replication_proportion = replicated_snps / total_gw_snps
  )